# Value Set Expansion examples

Value sets are enumerated permissible values (aka subsets) that are used for purposes such as filtering ontologies or defining
a subset of terms that are valid for data entry for a particular field.

Dynamical (extensional) value sets are value sets that are defined by a query (including boolean graph queries) rather than a fixed list of terms. Because handling dynamic value sets at runtime can be complex, a value set expander (materializer) can be used to turn a dynamic value set into a static one.

Value sets are found in formalisms such as LinkML and FHIR. Currently the OAK Value Set expander only supports LinkML value sets.

See:

 - https://linkml.io/linkml/schemas/enums.html#dynamic-enums


## VSKit command

Currently there is a command with a single subcommand (others may be added later) 

In [25]:
%%bash
vskit --help

Usage: vskit [OPTIONS] COMMAND [ARGS]...

  Run the ValueSet CLI.

Options:
  -v, --verbose
  -q, --quiet TEXT
  --help            Show this message and exit.

Commands:
  expand  Expand a value set.


In [3]:
%%bash
vskit expand --help

Usage: vskit expand [OPTIONS] [VALUE_SET_NAMES]...

  Expand a value set. EXPERIMENTAL.

  This will expand an *intentional value set* (aka *dynamic enum*), running a
  query against an ontology backend or backends to materialize the value set
  (permissible values).

  Currently the value set must be specified as LinkML, but in future this will
  be possible with other specifications such as FHIR ValueSet objects.

  Each expression in a dynamic enum has a *source ontology*, this is specified
  as a CURIE such as:

  - obo:mondo - bioregistry:wikidata

  These can be mapped to specific OAK selectors. By default, any obo prefix is
  mapped to the semsql implementation of that. You can use a configuration
  file to map to other backends, such as BioPortal or Wikidata. However, note
  that not all backends are capable of being able to render all value sets.

  Examples:

      vskit expand -c config.yaml -s schema.yaml -o expanded.yaml
      my_value_set1 my_value_set2

  Custom permissi

## Example Value Set

The test inputs folder has an example of multiple different kinds of value sets. 

In [4]:
%%bash
yq . ../../../tests/input/value_set_example.yaml

id: https://w3id.org/linkml/examples/enums
title: Dynamic Enums Example
name: dynamicenums-example
description: This demonstrates the use of dynamic enums
license: https://creativecommons.org/publicdomain/zero/1.0/
prefixes:
  linkml: https://w3id.org/linkml/
  ex: https://w3id.org/linkml/examples/enums/
  sh: https://w3id.org/shacl/
  bioregistry: https://bioregistry.io/registry/
  MONDO: http://purl.obolibrary.org/obo/MONDO_
  NCIT: http://purl.obolibrary.org/obo/NCIT_
  loinc: http://loinc.org/
default_prefix: ex
default_range: string
default_curi_maps:
  - semweb_context
emit_prefixes:
  - linkml
  - rdf
  - rdfs
  - xsd
  - owl
imports:
  - linkml:types
#==================================
# Classes                         #
#==================================
classes:
  HumanSample:
    slots:
      - name
      - disease
#==================================
# Slots                           #
#==================================
slots:
  name:
    range: string
  disease:
    range

### Example value set: membranes in GO

Let's examine the value set called GoMembrane:

In [7]:
%%bash
yq .enums.GoMembrane ../../../tests/input/value_set_example.yaml

pv_formula: CURIE
reachable_from:
  include_self: true
  source_ontology: obo:go
  source_nodes:
    - GO:0016020 ## membrane


You can see this is defined as a simple query that selects all terms that are a subclass of GO:0016020 (membrane);
i.e. an ontology *branch*. Other value sets are more complex involving boolean combinations.

## Configuration

Because the LinkML language is independent of OAK we need to bind the logical names used for vocabularies to OAK selector syntax. This is done in a configuration file.

In [14]:
CONFIG = """
resource_resolvers:
  obo:go:
    shorthand: sqlite:obo:go
""".strip()

In [15]:
with open("output/vskit-config.yaml", "w") as f:
    f.write(CONFIG)

In [16]:
%%bash
yq . output/vskit-config.yaml

resource_resolvers:
  obo:go:
    shorthand: sqlite:obo:go


## Expansion

Now we will expand the value set, into a new materialized file

In [19]:
%%bash
vskit expand --config output/vskit-config.yaml --schema ../../../tests/input/value_set_example.yaml GoMembrane -o output/GoMembrane.yaml

We can see the expanded value set below.

In [22]:
%%bash
yq .enums.GoMembrane.permissible_values output/GoMembrane.yaml | head -20

GO:0120201:
  text: GO:0120201
  description: Stack of disc membranes located inside a cone photoreceptor outer segment, and containing densely packed molecules of opsin photoreceptor proteins that traverse the lipid bilayer. Cone disc membranes arise as evaginations of the ciliary membrane during the development of the cone outer segment and remain contiguous with the ciliary membrane.
  meaning: GO:0120201
  title: cone photoreceptor disc membrane
GO:0060171:
  text: GO:0060171
  description: The portion of the plasma membrane surrounding a stereocilium.
  meaning: GO:0060171
  title: stereocilium membrane
GO:0042717:
  text: GO:0042717
  description: The lipid bilayer associated with a plasma membrane-derived chromatophore; surrounds chromatophores that form complete vesicles.
  meaning: GO:0042717
  title: plasma membrane-derived chromatophore membrane
GO:0035579:
  text: GO:0035579
  description: The lipid bilayer surrounding a specific granule, a granule with a membranous, tubula

note that technically the key value does not need repeated, but the default serialization is to include it.

## Customizing permissible values

We can use `--pv-syntax` to customize the permissible value serialization. A python-style format string is used to specify how the permissible values are serialized. The default is `{id}`.

In [23]:
%%bash
vskit expand --pv-syntax '{label} [{id}]' --config output/vskit-config.yaml --schema ../../../tests/input/value_set_example.yaml GoMembrane -o output/GoMembrane2.yaml

In [24]:
%%bash
yq .enums.GoMembrane.permissible_values output/GoMembrane2.yaml | head -20

cone photoreceptor disc membrane [GO:0120201]:
  text: cone photoreceptor disc membrane [GO:0120201]
  description: Stack of disc membranes located inside a cone photoreceptor outer segment, and containing densely packed molecules of opsin photoreceptor proteins that traverse the lipid bilayer. Cone disc membranes arise as evaginations of the ciliary membrane during the development of the cone outer segment and remain contiguous with the ciliary membrane.
  meaning: GO:0120201
  title: cone photoreceptor disc membrane
stereocilium membrane [GO:0060171]:
  text: stereocilium membrane [GO:0060171]
  description: The portion of the plasma membrane surrounding a stereocilium.
  meaning: GO:0060171
  title: stereocilium membrane
plasma membrane-derived chromatophore membrane [GO:0042717]:
  text: plasma membrane-derived chromatophore membrane [GO:0042717]
  description: The lipid bilayer associated with a plasma membrane-derived chromatophore; surrounds chromatophores that form complete ves